In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
# read it in to inspect it
with open('../dat/input.txt', 'r') as file:
    text = file.read()

In [3]:
print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


In [4]:
# Unique characters in the file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('Number of unique characters: ', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Number of unique characters:  65


In [5]:
# Mapping from characters to integers (naive approach)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda c: ''.join([itos[i] for i in c])

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [6]:
# Encode entire text
import torch

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
# Split data into training and validation
n = int(0.9*len(data))
train_data, val_data = data[:n], data[n:]

In [9]:
# Context length / block size (max)
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is {target}")

When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # random subset indices
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

import itertools
xb, yb = get_batch('train')
print("inputs: ", xb.shape)
print("targets: ", yb.shape)
print('------')

for b, t in itertools.product(range(batch_size), range(block_size)):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"When input is {context} the target is {target}")

inputs:  torch.Size([4, 8])
targets:  torch.Size([4, 8])
------
When input is tensor([24]) the target is 43
When input is tensor([24, 43]) the target is 58
When input is tensor([24, 43, 58]) the target is 5
When input is tensor([24, 43, 58,  5]) the target is 57
When input is tensor([24, 43, 58,  5, 57]) the target is 1
When input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
When input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
When input is tensor([44]) the target is 53
When input is tensor([44, 53]) the target is 56
When input is tensor([44, 53, 56]) the target is 1
When input is tensor([44, 53, 56,  1]) the target is 58
When input is tensor([44, 53, 56,  1, 58]) the target is 46
When input is tensor([44, 53, 56,  1, 58, 46]) the target is 39
When input is tensor([44, 53, 56,  1, 58, 46, 39]) the target is 58
When input is tensor([44, 53, 56,  1, 58, 46, 39, 58]) the target is 1
When input i

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

# * Simplest possible model: predict the next token from the current token

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # (B*T, C)
            targets = targets.view(B*T) # (B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) tensor of indices in the current context
        for _ in range(max_new_tokens):
            # get preds
            logits, loss = self(idx)
            # focus only on the last timestep
            logit = logits[: ,-1, :] # Becomes (B, 1, C) so (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logit, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=1) # (B, T+1) 
            # ! for a bigram model this is stupid as it does not use the history
        return idx

model = BigramLanguageModel(vocab_size)
out, loss = model(xb, yb)
print(out.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long) # Zeros (new line symbol) as first token
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [26]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    print(loss.item()) if steps % 1000 == 0 else None
    

2.509913206100464
2.410109519958496
2.4661755561828613
2.4207942485809326
2.384782314300537
2.3451786041259766
2.428402900695801
2.5643835067749023
2.5089166164398193
2.4304347038269043

MERNTI hy d ourenacoeio mo s BErow lou'ldenhous an:
ARD copanoery t.
Foue y lioupe w, t?
I're gar m!


In [29]:
# Test improvements
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))
    


BAs ty, sehoceamees mou TEOUCIEDifeainsts wir.
Rivexert tws kend sigsie d, thace sus,

SONThorleryo 
